# VacationPy
----

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
# Read csv file with city data
path_csv = os.path.join("..","OutputData","cities.csv")
city_df = pd.read_csv(path_csv)     

# Drop null values
city_df.dropna(inplace=True)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,praia da vitoria,38.7333,-27.0667,73.76,73,75,16.08,PT,1624107210
1,jamestown,42.0970,-79.2353,73.02,87,1,4.00,US,1624106971
2,mount isa,-20.7333,139.5000,47.97,49,5,0.00,AU,1624107212
3,kerema,-7.9631,145.7785,78.08,86,100,6.78,PG,1624107214
4,flinders,-34.5833,150.8552,56.21,77,100,10.00,AU,1624107216


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [24]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["Lat", "Lng"]].astype('float')

# Store 'Humidity' in humidity
humidity = city_df["Humidity"].astype('float')

In [25]:
# Create a Humidity Heatmap layer
fig = gmaps.figure(zoom_level = 2, center=(30,45))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
# Drop cities which does not fit weather conditions and create new dataframe
hotel_df = city_df.drop(city_df.loc[(city_df["Max Temp"] >= 80) |
                                         (city_df["Max Temp"] <= 70) |
                                         (city_df["Wind Speed"] >= 10) |
                                         (city_df["Cloudiness"] != 0)].index)
hotel_df.dropna(inplace=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,guaymas,27.9333,-110.9000,77.45,80,0,5.70,MX,1624107155,
13,kamenka,51.3223,42.7678,74.16,52,0,9.17,RU,1624107229,
16,cape town,-33.9258,18.4232,79.21,21,0,8.05,ZA,1624107233,
47,pervomayskiy,53.2485,40.2871,75.79,59,0,4.92,RU,1624107282,
51,albany,42.6001,-73.9662,75.63,74,0,5.44,US,1624107291,
116,east london,-33.0153,27.9116,72.54,46,0,8.05,ZA,1624107387,
293,shakawe,-18.3667,21.8500,79.29,20,0,9.28,BW,1624107669,
297,lahaina,20.8783,-156.6825,75.81,86,0,7.07,US,1624107627,
324,ozinki,51.1786,49.6769,78.44,35,0,5.70,RU,1624107717,
364,pocone,-16.2567,-56.6228,72.23,51,0,4.41,BR,1624107780,


In [28]:
# Store 'Lat' and 'Lng' into  locations 
locations = hotel_df[["Lat", "Lng"]].astype('float')

# Store 'Humidity' in humidity
humidity = hotel_df["Humidity"].astype('float')

# set up a parameters dictionary
params = {
    "location": "",
    "radius": 5000,
    "type": 'lodging',
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Iterate through each row and find hotel name
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location in each iteration
    params["location"] = f"{lat},{lng}"

    # run a request using our params dictionary
    hotel_data = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
    except:
        print("Hotel not found for " + hotel_df.loc[index, "City"]) 

hotel_df.drop(hotel_df.loc[hotel_df["Hotel Name"] == ""].index, inplace=True)
hotel_df

Hotel not found for kamenka
Hotel not found for pervomayskiy
Hotel not found for albany


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,guaymas,27.9333,-110.9000,77.45,80,0,5.70,MX,1624107155,Hotel Armida
16,cape town,-33.9258,18.4232,79.21,21,0,8.05,ZA,1624107233,Southern Sun Waterfront Cape Town
116,east london,-33.0153,27.9116,72.54,46,0,8.05,ZA,1624107387,Tu Casa
293,shakawe,-18.3667,21.8500,79.29,20,0,9.28,BW,1624107669,Ditlhapi Guest House
297,lahaina,20.8783,-156.6825,75.81,86,0,7.07,US,1624107627,Lahaina Shores Beach Resort
324,ozinki,51.1786,49.6769,78.44,35,0,5.70,RU,1624107717,Korolevskiy Dvor
364,pocone,-16.2567,-56.6228,72.23,51,0,4.41,BR,1624107780,Hotel e Churrascaria Pantaneira
403,nexo,55.0607,15.1306,70.23,74,0,9.60,DK,1624107839,Strandhotel Balka Søbad
446,mikhaylov,54.2292,39.0186,76.24,59,0,4.29,RU,1624107907,Gostinitsa
551,sao filipe,14.8961,-24.4956,74.55,73,0,5.77,CV,1624108063,Hotel Xaguate


In [29]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Store the DataFrame Row
locations = hotel_df[["Lat", "Lng"]]

# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(locations,
                             info_box_content=hotel_info)

In [30]:
# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))